## Import code packages

In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import selenium.webdriver.chrome.service as service 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.options import Options
import getpass
import time
import pprint
import random
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime
vaccine_found = False

### Enter Walgreens credentials

In [2]:
username_to_enter = input("Please enter your walgreens username: ") 
password_to_enter = getpass.getpass("Please enter your walgreens password: ")

Please enter your walgreens username:  samuel.piecz@gmail.com
Please enter your walgreens password:  ··············


### Enter Gmail credentials

In [3]:
gmail_user = input("Please enter your email username: ")
gmail_password = getpass.getpass("Please enter your email password: ")

Please enter your email username:  samuel.piecz@gmail.com
Please enter your email password:  ················


### Enter ZIP code

In [4]:
# Define zip code
myZipCode = input('Please enter your zip code: ')

Please enter your zip code:  Woodstock, IL


## Define function to send email for use later

In [223]:
def send_email(receiver_addresses = ['samuel.piecz@gmail.com','tony.piecz@gmail.com', 'ponypreacher@gmail.com']):
    mail_content = '''Login to register for your vaccine.'''

    #The mail addresses and password
    sender_address = gmail_user
    sender_pass = gmail_password

    for receiver_address in receiver_addresses:

        #Setup the MIME
        message = MIMEMultipart()
        message['From'] = sender_address
        message['To'] = receiver_address
        message['Subject'] = 'Register for COVID Vaccine Appointment Now'   #The subject line

        #The body and the attachments for the mail
        message.attach(MIMEText(mail_content, 'plain'))

        #Create SMTP session for sending the mail
        session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
        session.starttls() #enable security
        session.login(sender_address, sender_pass) #login with mail_id and password
        text = message.as_string()

        # Send Mail
        session.sendmail(sender_address, receiver_address, text)
        session.quit()
        print('Mail Sent')

## Define login function

In [185]:
def login():
    driver.get("https://www.walgreens.com/login.jsp")

    driver.get_screenshot_as_file("test1.png")

    login = driver.find_element_by_xpath('//*[@id="user_name"]')
    login.clear()
    login.send_keys(username_to_enter)
    time.sleep(1)
    password = driver.find_element_by_xpath('//*[@id="user_password"]')
    password.clear()
    password.send_keys(password_to_enter)
    time.sleep(1)
    show_pass = driver.find_element_by_xpath('//*[@id="ShowCharacter"]')
    show_pass.click()
    time.sleep(1)
    sign_in = driver.find_element_by_xpath('//*[@id="submit_btn"]')
    sign_in.click()
    time.sleep(1)
    driver.get_screenshot_as_file("test2.png")

## Define function to handle verification email

In [143]:
def verify(gmail_user, gmail_password):
    # verify code
    email_verify = driver.find_element_by_xpath('//*[@id="radio-email"]')
    email_verify.click()
    send_verification = driver.find_element_by_xpath('//*[@id="btn_send_verify_code"]')
    send_verification.click()
    time.sleep(5)

    auth_code = ""
    # TODO grab verification from gmail
    import email
    import imaplib
    import mailbox

    EMAIL_ACCOUNT = gmail_user
    PASSWORD = gmail_password
    folder = "walgreens"

    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    mail.login(EMAIL_ACCOUNT, PASSWORD)
    mail.list()
    mail.select(folder)
    result, data = mail.uid('search', None, "ALL") # (ALL/UNSEEN)

    latest_email_uid = data[0].split()[-1]
    result, email_data = mail.uid('fetch', latest_email_uid, '(RFC822)')
    # result, email_data = conn.store(num,'-FLAGS','\\Seen')
    # this might work to set flag to seen, if it doesn't already
    raw_email = email_data[0][1]
    raw_email_string = raw_email.decode('utf-8')
    email_message = email.message_from_string(raw_email_string)
    subject = str(email.header.make_header(email.header.decode_header(email_message['Subject'])))
    print(subject.split(":")[1].strip())
    auth_code = subject.split(":")[1].strip()

    enter_verification = driver.find_element_by_xpath('//*[@id="auth-code"]')
    enter_verification.send_keys(auth_code)
    
    submit_verification = driver.find_element_by_xpath('//*[@id="validate_security_answer"]')
    submit_verification.click()
    
    driver.get_screenshot_as_file("test6.png")


## Define function to see if vaccine is available

In [225]:
def checkAvailability():
    driver.get("https://www.walgreens.com/findcare/vaccination/covid-19?ban=covid_vaccine_landing_schedule")
    
    driver.get_screenshot_as_file("test3.png")
    
    # Click button
    scheduleNewAppointment = driver.find_element_by_xpath('/html/body/div[2]/section/section/section[2]/div/section/section/section[2]/a')
    scheduleNewAppointment.click()
    
    driver.get_screenshot_as_file("test4.png")

    # Enter zip code
    zipCodeInput = driver.find_element_by_xpath('//*[@id="inputLocation"]')
    time.sleep(2)
    zipCodeInput.clear()
    zipCodeInput.send_keys(myZipCode)
    
    # Click search button
    searchZipButton = driver.find_element_by_xpath('/html/body/div[2]/div/div/section/section/section/section/section/section/section[2]/div/span/button')
    searchZipButton.click()
    
    driver.get_screenshot_as_file("test5.png")

    
    # Check if appointments available or not
    popupText = driver.find_element_by_xpath('//*[@id="wag-body-main-container"]/section/section/section/section/div/a/span[2]/p')
    
    # Send Email or display no appointments available
    if popupText.text == 'Appointments unavailable':
        print("No vaccines available. {}".format(datetime.now()))
        return False
    else:
        send_email()
        return True

## Define function to instantiate the web browser

In [231]:
# def start_driver(headless=False):
    # sets options to run in headless
chrome_options = Options()
chrome_options.add_argument('--start-maximized')
# if headless:
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# sets driver
driver = webdriver.Chrome("./chromedriver", options=chrome_options)
driver.implicitly_wait(15)

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

{'identifier': '2'}

## Scrape Walgreens

In [219]:
# login()
# header = driver.find_element_by_xpath('//*[@id="page-content"]/div[2]/div[1]/h1')

# if header.text == "Your Account":
#     checkAvailability()
# else:
#     verify()

In [232]:
while True:
    thing = checkAvailability()
    if thing == True:
        break
    time.sleep(60)

No vaccines available. 2021-03-14 23:34:12.680239
No vaccines available. 2021-03-14 23:35:21.760907
No vaccines available. 2021-03-14 23:36:31.008973
No vaccines available. 2021-03-14 23:37:41.820633
No vaccines available. 2021-03-14 23:38:50.934578
No vaccines available. 2021-03-14 23:39:59.581973
No vaccines available. 2021-03-14 23:41:08.030340
No vaccines available. 2021-03-14 23:42:16.371900
No vaccines available. 2021-03-14 23:43:24.604409
No vaccines available. 2021-03-14 23:44:33.621833
No vaccines available. 2021-03-14 23:45:41.351915
No vaccines available. 2021-03-14 23:46:49.701532
No vaccines available. 2021-03-14 23:47:57.841254


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="inputLocation"]"}
  (Session info: chrome=89.0.4389.82)
